In [34]:
# import thu vien
import os

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import  Model

from PIL import Image
import pickle
import numpy as np
import matplotlib.pyplot as plt
import math
import streamlit as st
import time
from scipy.spatial.distance import cdist

In [6]:
path_dataset= 'anhhoa_resize'

In [7]:
# Ham tao model
def get_extract_model():
    vgg16_model = VGG16(weights="imagenet")
    extract_model = Model(inputs=vgg16_model.inputs, outputs = vgg16_model.get_layer("fc1").output)
    return extract_model

In [8]:
# Ham tien xu ly, chuyen doi hinh anh thanh tensor
def image_preprocess(img):
    img = img.resize((224,224))
    img = img.convert("RGB")
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

def extract_vector(model, image_path):
    print("Xu ly : ", image_path)
    img = Image.open(image_path)
    img_tensor = image_preprocess(img)

    # Trich dac trung
    vector = model.predict(img_tensor)[0]
    # Chuan hoa vector = chia chia L2 norm (tu google search)
    vector = vector / np.linalg.norm(vector)
    return vector

In [27]:
search_img = "hoahong02.jpg"

In [32]:
#khoi tao model
model = get_extract_model()

#trich chon dac trung anh search
search_vector = extract_vector(model, search_img)

#load model
vectors = pickle.load(open("vectors.pkl","rb"))
paths = pickle.load(open("paths.pkl","rb"))

Xu ly :  hoahong02.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step


In [35]:
# Tính khoảng cách giữa vector tìm kiếm và các vector trong tập dữ liệu
distance = np.linalg.norm(vectors - search_vector, axis=1)

# Chọn ra K ảnh có khoảng cách ngắn nhất
K = 3
ids = np.argsort(distance)[:K]
nearest_images = [(paths[id], distance[id]) for id in ids]

# Tạo giao diện Streamlit
st.title("Top 3 Most Similar Images")
st.write("Search Image:")
st.image(search_img)

for idx, (path, dist) in enumerate(nearest_images):
    st.write(f"Similar Image {idx+1}:")
    st.image(Image.open(path))
    st.write(f"Distance: {dist:.2f}")


2024-06-07 15:41:42.985 
  command:

    streamlit run C:\Users\Admin\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py [ARGUMENTS]
